In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [71]:
df = pd.read_csv('indexData.csv')

## Missing Value

#### Various variants of Missing Values:
1. Missing completely at random(MCAR) – Data is missing due to mishandling of data, Can’t be predicted from rest of the variables. Example : Data lost during ETL process, (Similar Case Imputation might not work here)
2. Missing at random(MAR) – Some fields which are not relevant to a certain sample of population would be missing. Example : whether a group of older people (age > 45) will be using ZEE5,Prime Video or not. (Similar Case Imputation might work here)
3. Missing not at random(MNAR) – If we have data that is not missing randomly but had been intentionally left empty. Example : Sick people leaving a field which asks for alcohol (Yes/No), would rather choose to leave the field empty. (Tricky Situation might have to evaluate different options while filling missing data).

#### How to handle Missing data (Approach) : 
1. Find the columns where data is missing with respective percentage.
2. If less than 20% data is missing, First try to find the reason why the data is missing, they might be missing due to a single reason or due to different reasons, But if more than 20% data is missing for any attribute, we might need to discuss with stakeholders to check if we can drop the column or we can somehow enrich that attribute.
3. Post finding the reason of missing, check if the data can be enriched from source (If the data was missing due to mishandling).
4. Post step 3, If we have missing values left, we can use methods listed in the next slide.

#### Number of Missing values in absolute numbers 

In [72]:
df.isna().sum()

Index           0
Date            0
Open         2204
High         2204
Low          2204
Close        2204
Adj Close    2204
Volume       2204
dtype: int64

#### Number of missing value in %

In [73]:
(df.isna().sum()/df.shape[0]) * 100

Index        0.00000
Date         0.00000
Open         1.95986
High         1.95986
Low          1.95986
Close        1.95986
Adj Close    1.95986
Volume       1.95986
dtype: float64

#### Number of fill value in % or Fill Rate

In [74]:
((1 - df.isna().sum()/df.shape[0]) * 100)

Index        100.00000
Date         100.00000
Open          98.04014
High          98.04014
Low           98.04014
Close         98.04014
Adj Close     98.04014
Volume        98.04014
dtype: float64

##### Conclusion : As we see we have almost 98+ %age of data present, we need not drop any attribute . i.e, Dropping of rows/columns is not needed. 

## Outlier Removal

In [75]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [88]:
df1 = df.copy()
df2 = df.copy()

In [89]:
df1 = remove_outlier(df1,'Open')
df1 = remove_outlier(df1,'High')
df1 = remove_outlier(df1,'Low')
df1 = remove_outlier(df1,'Close')
df1 = remove_outlier(df1,'Adj Close')
df1 = remove_outlier(df1,'Volume')

In [90]:
print(f"Number of records dropped which were considered as outlier in IQR Based Approach : {df.shape[0] - df1.shape[0]}")

Number of records dropped which were considered as outlier in IQR Based Approach : 27797


##### Several Other Methods include : 

1. Outlier Removal using Pyod(Python Package) 
2. Z-Score Method of Outlier Filtering : Filter out all the data points which are beyond (Mean     3* Standard Deviation) , As we expect 99.7% records failing in this range. i.e., Z-Score > 3


#### Single Imputation Method : 

1. Imputation with Mean – If the distribution is normal.
2. Imputation with Median – If the distribution is not normal
3. Imputation with Mode – If the data is categorical in nature

#### creating a dataset where we have missing categorical data

In [91]:
df.loc[50:55,'Index'] = np.nan

In [92]:
df.isna().sum()

Index        6
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

#### Imputation with Mode

In [93]:
df['Index'] = df['Index'].fillna(df['Index'].mode()[0])

In [94]:
df.isna().sum()

Index        0
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

#### Check if we can replace with Mean, As we have removed outliers in df1, we can use df1 to replace with mean

In [95]:
missing_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

for col in missing_cols:
    df1[col].fillna(df1[col].mean(),inplace=True)

In [96]:
df1.isna().sum()

Index        0
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

#### But for Df we did not remove the outliers , so we can't replace with Mean but with Median

In [97]:
for col in missing_cols:
    df[col].fillna(df[col].median(),inplace=True)

In [98]:
df.isna().sum()

Index        0
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

#### For stock price missing data, we recommend forward fill & Backward Fill

In [100]:
df2 = df2.ffill(axis = 1)

In [101]:
df2.isna().sum()

Index        0
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

#### Dropping duplicates

In [102]:
# Considering all the columns and dropping duplicates
df2.drop_duplicates(inplace=True)

In [103]:
# Considering only few columns and dropping duplicates
df2.drop_duplicates(subset=['Index','Date'],keep='first',inplace=True)

#### Data Format Change

In [142]:
#Inserting a erroneous date
df.loc[33,'Date'] = '202212032'

In [143]:
# find the records which are not matching %Y-%m-%d datetime format
indexes = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce').isna()

In [144]:
df.loc[indexes,'Date']

33    202212032
Name: Date, dtype: object

In [145]:
df.loc[indexes,'Date'] = pd.to_datetime(datetime(2022,12,3), format='%Y-%m-%d')

In [147]:
indexes = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce').isna()

In [148]:
df.loc[indexes,'Date']

Series([], Name: Date, dtype: object)